In [1]:
!pip install insightface --upgrade --quiet
!pip install onnxruntime-gpu --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 92.2 MB/s eta 0:00:00:00:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
mkl-fft 1.3.8 requires numpy<1.27.0,>=1.26.4, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you 

In [2]:

# Vérifier la version du driver NVIDIA et du GPU
!nvidia-smi

# Vérifier la version de CUDA installée
!nvcc --version

Tue Dec  9 09:06:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# Vérifier la version de cuDNN
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2
!sudo ln -sf /usr/local/cuda/lib64/libcublasLt.so.12 /usr/local/cuda/lib64/libcublasLt.so.11



#define CUDNN_MAJOR 9
#define CUDNN_MINOR 2
#define CUDNN_PATCHLEVEL 1
--
#define CUDNN_VERSION (CUDNN_MAJOR * 10000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [4]:
ls -l /usr/local/cuda/lib64/libcublasLt.so.11


lrwxrwxrwx 1 root root 39 Dec  9 09:06 /usr/local/cuda/lib64/libcublasLt.so.11 -> /usr/local/cuda/lib64/libcublasLt.so.12


In [5]:
!pip install onnxruntime-gpu==1.20.0  # compatible CUDA 12
import onnxruntime as ort
print("Available providers:", ort.get_available_providers())
!pip install insightface onnxruntime-gpu tqdm
!pip uninstall -y matplotlib insightface onnxruntime onnxruntime-gpu
!pip install matplotlib==3.8.4 insightface onnxruntime-gpu tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.5/291.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: onnxruntime-gpu
    Found existing installation: onnxruntime-gpu 1.23.2
    Uninstalling onnxruntime-gpu-1.23.2:
      Successfully uninstalled onnxruntime-gpu-1.23.2
Available providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
Found existing installation: matplotlib 3.7.2
Uninstalling matplotlib-3.7.2:
  Successfully uninstalled matplotlib-3.7.2
Found existing installation: insightface 0.7.3
Uninstalling insightface-0.7.3:
  Successfully uninstalled insightface-0.7.3
Found existing installation: onnxruntime-gpu 1.20.0
Uninstalling onnxruntime-gpu-1.20.0:
  Successfully uninstalled onnxruntime-gpu-1.20.0
  Using cached insightface-0.7.3-cp311-cp311-linux_x86_64.whl
  Using cached onnxruntime_gpu-1.23.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB

In [ ]:
!pip install insightface


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# -------------------------------
# 🔹 Initialisation InsightFace
# -------------------------------

import os
import cv2
import shutil
import pandas as pd
from tqdm import tqdm
from insightface.app import FaceAnalysis

# -------------------------------
# ⚙️ Paramètres
# -------------------------------
dataset_root = "/kaggle/input/cacd-filtered-dataset/cacd_split/cacd_split"
output_root = "/kaggle/working/cacd_cleaned_dataset_fast"
csv_file = os.path.join(output_root, "cacd_cleaned_dataset_fast.csv")

# Tranches d'âge pour StarGAN
age_groups = {
    "trainA": (0, 24),    # jeunes <25
    "trainB": (25, 49),   # adultes 25-49
    "trainC": (50, 150)   # vieux ≥50
}

os.makedirs(output_root, exist_ok=True)
for group in age_groups.keys():
    os.makedirs(os.path.join(output_root, group), exist_ok=True)


print("🚀 Chargement du modèle InsightFace Age...")
app = FaceAnalysis(providers=['CUDAExecutionProvider'])  # GPU
app.prepare(ctx_id=0)  # ctx_id=0 pour GPU 0
print("✅ Modèle prêt !")

# -------------------------------
# 🔹 Fonction de traitement d'une image
# -------------------------------
def process_image(star_folder, img_file):
    folder_path = os.path.join(dataset_root, star_folder)
    img_path = os.path.join(folder_path, img_file)

    img = cv2.imread(img_path)
    if img is None:
        return None

    faces = app.get(img)
    if not faces:
        return None  # pas de visage détecté

    age_pred = int(faces[0].age)

    # Déterminer tranche d'âge
    for group, (min_age, max_age) in age_groups.items():
        if min_age <= age_pred <= max_age:
            group_folder = os.path.join(output_root, group, star_folder)
            os.makedirs(group_folder, exist_ok=True)
            new_name = f"{age_pred}_{star_folder}_{img_file.split('_')[-1]}"
            new_path = os.path.join(group_folder, new_name)
            shutil.copy(img_path, new_path)
            return {
                "star": star_folder,
                "original_image": img_file,
                "predicted_age": age_pred,
                "new_name": new_name,
                "age_group": group
            }
    return None

# -------------------------------
# 🔹 Préparer toutes les images
# -------------------------------
all_images = []
for star_folder in os.listdir(dataset_root):
    folder_path = os.path.join(dataset_root, star_folder)
    if not os.path.isdir(folder_path):
        continue
    for img_file in os.listdir(folder_path):
        if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
            all_images.append((star_folder, img_file))

print(f"📊 Total images à traiter : {len(all_images)}")



# -------------------------------
# 🔹 Traitement séquentiel (GPU)
# -------------------------------
results = []
for star_folder, img_file in tqdm(all_images, total=len(all_images)):
    r = process_image(star_folder, img_file)
    if r is not None:
        results.append(r)

# -------------------------------
# 🔹 Génération du CSV
# -------------------------------
df = pd.DataFrame(results)
df.to_csv(csv_file, index=False)
print(f"✅ Dataset nettoyé et CSV créé : {csv_file}")

# -------------------------------
# 🔹 Stats rapides
# -------------------------------
print("\n📊 Stats par tranche d'âge :")
for group in age_groups.keys():
    count = len(df[df['age_group'] == group])
    print(f"{group}: {count} images")

🚀 Chargement du modèle InsightFace Age...
download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:02<00:00, 104013.49KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

100%|██████████| 154915/154915 [1:43:59<00:00, 24.83it/s]  


✅ Dataset nettoyé et CSV créé : /kaggle/working/cacd_cleaned_dataset_fast/cacd_cleaned_dataset_fast.csv

📊 Stats par tranche d'âge :
trainA: 4774 images
trainB: 103020 images
trainC: 45653 images


In [2]:

# ==============================================
# 🔹 Filtrer les stars présentes dans les 3 tranches d'âge
# ==============================================

import pandas as pd
import os
import shutil
from tqdm import tqdm

# -------------------------------
# ⚙️ Paramètres
# -------------------------------
input_root = "/kaggle/working/cacd_cleaned_dataset_fast"
output_root = "/kaggle/working/cacd_complete_stars_dataset"
input_csv = os.path.join(input_root, "cacd_cleaned_dataset_fast.csv")
output_csv = os.path.join(output_root, "cacd_complete_stars_dataset.csv")

age_groups = ["trainA", "trainB", "trainC"]

os.makedirs(output_root, exist_ok=True)
for group in age_groups:
    os.makedirs(os.path.join(output_root, group), exist_ok=True)

# -------------------------------
# 🔹 Charger le CSV existant
# -------------------------------
df = pd.read_csv(input_csv)

# -------------------------------
# 🔹 Identifier les stars complètes
# -------------------------------
stars_complete = []

for star, group_df in df.groupby('star'):
    star_groups = set(group_df['age_group'].unique())
    if all(g in star_groups for g in age_groups):
        stars_complete.append(star)

print(f"⭐ Nombre de stars avec images dans les 3 tranches : {len(stars_complete)}")

# -------------------------------
# 🔹 Filtrer le DataFrame
# -------------------------------
df_filtered = df[df['star'].isin(stars_complete)].copy()

# -------------------------------
# 🔹 Copier les images dans le nouveau dossier
# -------------------------------
for _, row in tqdm(df_filtered.iterrows(), total=len(df_filtered), desc="Copie images"):
    src_path = os.path.join(input_root, row['age_group'], row['star'], row['new_name'])
    dest_folder = os.path.join(output_root, row['age_group'], row['star'])
    os.makedirs(dest_folder, exist_ok=True)
    dest_path = os.path.join(dest_folder, row['new_name'])
    if not os.path.exists(dest_path):
        shutil.copy(src_path, dest_path)

# -------------------------------
# 🔹 Enregistrer le CSV final
# -------------------------------
df_filtered.to_csv(output_csv, index=False)
print(f"✅ CSV des stars complètes créé : {output_csv}")

# -------------------------------
# 🔹 Stats rapides
# -------------------------------
print("\n📊 Stats par tranche d'âge pour stars complètes :")
for group in age_groups:
    count = len(df_filtered[df_filtered['age_group'] == group])
    print(f"{group}: {count} images")

⭐ Nombre de stars avec images dans les 3 tranches : 1012


Copie images: 100%|██████████| 77685/77685 [00:16<00:00, 4612.19it/s]


✅ CSV des stars complètes créé : /kaggle/working/cacd_complete_stars_dataset/cacd_complete_stars_dataset.csv

📊 Stats par tranche d'âge pour stars complètes :
trainA: 3864 images
trainB: 57037 images
trainC: 16784 images


In [3]:
import os
from PIL import Image

root = "/kaggle/working/cacd_complete_stars_dataset"
splits = ['trainA','trainB','trainC']

corrupted = []

for sp in splits:
    sp_path = os.path.join(root, sp)
    if not os.path.isdir(sp_path):
        continue
    for sub in os.listdir(sp_path):
        sub_path = os.path.join(sp_path, sub)
        if not os.path.isdir(sub_path):
            continue
        for f in os.listdir(sub_path):
            f_path = os.path.join(sub_path, f)
            try:
                img = Image.open(f_path)
                img.verify()  # Vérifie si l'image est correcte
            except:
                corrupted.append(f_path)

print(f"Images corrompues trouvées: {len(corrupted)}")
for f in corrupted:
    os.remove(f)
    print(f"Supprimée: {f}")

Images corrompues trouvées: 0


In [4]:

import shutil
import os

samples_dir = "/kaggle/working/outputs/samples"

if os.path.exists(samples_dir):
    # Supprime tout le contenu du dossier samples
    for f in os.listdir(samples_dir):
        f_path = os.path.join(samples_dir, f)
        if os.path.isdir(f_path):
            shutil.rmtree(f_path)
        else:
            os.remove(f_path)
    print(f"✅ Tout le contenu de {samples_dir} a été supprimé")
else:
    print(f"⚠️ Le dossier {samples_dir} n'existe pas")





⚠️ Le dossier /kaggle/working/outputs/samples n'existe pas


In [ ]:
# ====================== STAR GAN V2 - Version OPTIMISÉE (Netteté + Cheveux Gris + 40k iters) ======================
import os, random, glob
from collections import Counter
from tqdm import tqdm
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.utils import save_image
from torch.amp import autocast, GradScaler

# ====================== CONFIG OPTIMISÉE ======================
CFG = {
    "data_root": "/kaggle/working/cacd_complete_stars_dataset",
    "out_dir": "/kaggle/working/starganv2_optimized",
    "iters": 40000,  # +10k
    "img_size": 128,
    "batch_size": 28,
    "num_workers": 4,
    "lr": 1.5e-4,
    "latent_dim": 16,
    "style_dim": 64,
    "n_domains": 3,  # 0=jeune, 1=adulte, 2=âgé
    "n_res": 4,
    "lambda_cyc": 5.0,
    "lambda_cls": 1.0,
    "lambda_sty": 0.1,
    "lambda_reg": 0.0001,
    "sample_every": 500,
    "save_every": 5000,
    "seed": 42,
    "sharpen_alpha": 0.3,
    "gray_hair_intensity": 0.3
}

# ====================== SEED ======================
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(CFG["seed"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(CFG["out_dir"] + "/samples", exist_ok=True)
os.makedirs(CFG["out_dir"] + "/checkpoints", exist_ok=True)

# ====================== UTILS ======================
def sharpen(img_tensor, alpha=0.3):
    kernel = torch.tensor([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=torch.float32, device=img_tensor.device)
    kernel = kernel.unsqueeze(0).unsqueeze(0)
    img_sharp = F.conv2d(img_tensor.unsqueeze(0), kernel.repeat(3,1,1,1), padding=1, groups=3)
    return torch.clamp((1-alpha)*img_tensor + alpha*img_sharp.squeeze(0), -1, 1)

def add_gray_hair(img_tensor, domain, intensity=0.3):
    if domain.item() == 2:  # domaine ÂGÉ
        gray = img_tensor.mean(dim=1, keepdim=True)
        img_tensor = img_tensor * (1-intensity) + gray * intensity
    return img_tensor

# ====================== DATASET ======================
class AgeDataset(Dataset):
    def __init__(self, root, img_size=128):
        self.samples = []
        self.domain_names = ["trainA", "trainB", "trainC"]
        for d_idx, folder in enumerate(self.domain_names):
            path = os.path.join(root, folder)
            if not os.path.isdir(path): continue
            for person in os.listdir(path):
                person_path = os.path.join(path, person)
                if not os.path.isdir(person_path): continue
                for file in glob.glob(os.path.join(person_path, "*.*")):
                    if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
                        self.samples.append((file, d_idx))
        print(f"Dataset chargé : {len(self.samples)} images")
        print(f"Répartition des domaines: {Counter([d for _,d in self.samples])}")

        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, domain = self.samples[idx]
        try:
            img = Image.open(path).convert("RGB")
        except:
            img = Image.new("RGB", (CFG["img_size"], CFG["img_size"]))
        return self.transform(img), domain, path

# ====================== MODELS ======================
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.normal_(m.weight, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.constant_(m.bias, 0)

class AdaIN(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.norm = nn.InstanceNorm2d(channels, affine=False)
        self.style = nn.Linear(style_dim, channels * 2)
    def forward(self, x, s):
        style = self.style(s)
        gamma, beta = style.chunk(2, 1)
        gamma = gamma.unsqueeze(-1).unsqueeze(-1)
        beta = beta.unsqueeze(-1).unsqueeze(-1)
        return gamma * self.norm(x) + beta

class ResBlock(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.conv2 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.adain1 = AdaIN(channels, style_dim)
        self.adain2 = AdaIN(channels, style_dim)
    def forward(self, x, s):
        h = self.conv1(x)
        h = self.adain1(h, s)
        h = F.relu(h, inplace=True)
        h = self.conv2(h)
        h = self.adain2(h, s)
        return x + h

class Generator(nn.Module):
    def __init__(self, style_dim=64, n_res=4):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv2d(3, 64, 7, 1, 3, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.ReLU(True),
        )
        self.res = nn.ModuleList([ResBlock(256, style_dim) for _ in range(n_res)])
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 3, 7, 1, 3), nn.Tanh()
        )
    def forward(self, x, s):
        h = self.enc(x)
        for block in self.res:
            h = block(h, s)
        return self.dec(h)

class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, n_domains=3):
        super().__init__()
        self.n_domains = n_domains
        self.shared = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(True),
            nn.Linear(256, 256), nn.ReLU(True),
        )
        self.branches = nn.ModuleList([
            nn.Sequential(
                nn.Linear(256, 256), nn.ReLU(True),
                nn.Linear(256, style_dim)
            ) for _ in range(n_domains)
        ])
    def forward(self, z, domain):
        h = self.shared(z)
        styles = [branch(h) for branch in self.branches]
        styles = torch.stack(styles, dim=1)
        batch_size = z.size(0)
        return styles[torch.arange(batch_size), domain]

class StyleEncoder(nn.Module):
    def __init__(self, style_dim=64, n_domains=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 7, 1, 3), nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1), nn.ReLU(True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.style = nn.Linear(256, style_dim)
        self.classifier = nn.Linear(256, n_domains)
    def forward(self, x):
        h = self.net(x).view(x.size(0), -1)
        style = self.style(h)
        cls_output = self.classifier(h)
        return style, cls_output

class Discriminator(nn.Module):
    def __init__(self, n_domains=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(512, 1024, 4, 1, 0), nn.LeakyReLU(0.2, True),
        )
        self.src = nn.Conv2d(1024, 1, 3, 1, 1)
        self.cls = nn.Conv2d(1024, n_domains, 3, 1, 1)
    def forward(self, x):
        h = self.main(x)
        src = self.src(h).view(x.size(0), -1)
        cls = self.cls(h).mean([2,3])
        return src, cls

# ====================== DATA & MODELS ======================
dataset = AgeDataset(CFG["data_root"])
counts = Counter([d for _,d in dataset.samples])
weights = [1.0 / counts[d] for _,d in dataset.samples]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
loader = DataLoader(dataset, batch_size=CFG["batch_size"], sampler=sampler,
                    num_workers=CFG["num_workers"], drop_last=True,
                    pin_memory=True, persistent_workers=True)

G = Generator(CFG["style_dim"], CFG["n_res"]).to(device)
F_map = MappingNetwork(CFG["latent_dim"], CFG["style_dim"], CFG["n_domains"]).to(device)
F_style = StyleEncoder(CFG["style_dim"], CFG["n_domains"]).to(device)
D = Discriminator(CFG["n_domains"]).to(device)

G.apply(weights_init); F_map.apply(weights_init); F_style.apply(weights_init); D.apply(weights_init)

opt_g = torch.optim.Adam(list(G.parameters()) + list(F_map.parameters()) + list(F_style.parameters()), 
                         lr=CFG["lr"], betas=(0.5, 0.999))
opt_d = torch.optim.Adam(D.parameters(), lr=CFG["lr"], betas=(0.5, 0.999))
scaler_g = GradScaler()
scaler_d = GradScaler()

# ====================== TRAINING LOOP ======================
pbar = tqdm(total=CFG["iters"])
domain_names = ["JEUNE", "ADULTE", "ÂGÉ"]
loader_iter = iter(loader)

for it in range(1, CFG["iters"] + 1):
    try:
        real, real_domain, _ = next(loader_iter)
    except StopIteration:
        loader_iter = iter(loader)
        real, real_domain, _ = next(loader_iter)

    real = real.to(device, non_blocking=True)
    real_domain = real_domain.to(device, non_blocking=True)
    b = real.size(0)

    # --- reste de la boucle unchanged ---

    # --- Discriminator ---
    opt_d.zero_grad()
    with autocast('cuda'):
        tgt_domain = torch.randint(0, CFG["n_domains"], (b,), device=device)
        z = torch.randn(b, CFG["latent_dim"], device=device)
        style = F_map(z, tgt_domain)
        fake = G(real, style).detach()

        real_src, real_cls = D(real)
        fake_src, _ = D(fake)

        loss_d_adv = F.softplus(-real_src).mean() + F.softplus(fake_src).mean()
        loss_d_cls = F.cross_entropy(real_cls, real_domain)
        loss_d = loss_d_adv + CFG["lambda_cls"] * loss_d_cls

    scaler_d.scale(loss_d).backward()
    scaler_d.step(opt_d)
    scaler_d.update()

    # --- Generator ---
    opt_g.zero_grad()
    with autocast('cuda'):
        tgt_domain = torch.randint(0, CFG["n_domains"], (b,), device=device)
        z = torch.randn(b, CFG["latent_dim"], device=device)
        style = F_map(z, tgt_domain)
        fake = G(real, style)
        fake_src, fake_cls = D(fake)

        loss_g_adv = -fake_src.mean()
        loss_g_cls = F.cross_entropy(fake_cls, tgt_domain)

        style_orig, _ = F_style(real)
        rec = G(fake, style_orig)
        loss_cyc = F.l1_loss(rec, real)

        z2 = torch.randn(b, CFG["latent_dim"], device=device)
        style2 = F_map(z2, tgt_domain)
        fake2 = G(real, style2)
        loss_sty = -F.l1_loss(fake, fake2) * CFG["lambda_sty"]

        style_reg = F.l1_loss(style, torch.zeros_like(style)) * CFG["lambda_reg"]

        loss_g = (loss_g_adv + CFG["lambda_cls"] * loss_g_cls + 
                 CFG["lambda_cyc"] * loss_cyc + loss_sty + style_reg)

    scaler_g.scale(loss_g).backward()
    scaler_g.step(opt_g)
    scaler_g.update()

    # --- SAMPLING DIAGNOSTIQUE ---
    if it % CFG["sample_every"] == 0:
        G.eval(); F_map.eval(); F_style.eval()
        with torch.no_grad():
            ref_images, ref_domains_list = [], []
            for domain in range(CFG["n_domains"]):
                domain_mask = (real_domain == domain)
                if domain_mask.sum() > 0:
                    idx = torch.where(domain_mask)[0][0]
                    ref_images.append(real[idx])
                    ref_domains_list.append(domain)
            
            if len(ref_images) >= 2:
                current_ref = torch.stack(ref_images[:min(4, len(ref_images))])
                ref_domains = torch.tensor(ref_domains_list[:current_ref.size(0)], device=device)
                
                outs = [current_ref.cpu()]
                
                for target_domain in range(CFG["n_domains"]):
                    tgt_domains = torch.full((current_ref.size(0),), target_domain, device=device)
                    z = torch.randn(current_ref.size(0), CFG["latent_dim"], device=device)
                    s = F_map(z, tgt_domains)
                    gen = G(current_ref, s)
                    # Appliquer cheveux gris et netteté
                    for i in range(gen.size(0)):
                        gen[i] = add_gray_hair(gen[i], tgt_domains[i], CFG["gray_hair_intensity"])
                        gen[i] = sharpen(gen[i], CFG["sharpen_alpha"])
                    outs.append(gen.cpu())

                grid = torch.cat(outs, dim=0)
                save_image(grid * 0.5 + 0.5, 
                          f"{CFG['out_dir']}/samples/{it:06d}.jpg", 
                          nrow=current_ref.size(0), padding=4, pad_value=1)
                
                ref_names = [domain_names[d] for d in ref_domains.cpu()]
                print(f"\n🎭 Sample {it}: {ref_names} → [JEUNE, ADULTE, ÂGÉ]")

        G.train(); F_map.train(); F_style.train()

       # --- Checkpoint (poids seulement pendant le training) ---
    if it % CFG["save_every"] == 0:
        torch.save({
            "G": G.state_dict(),
            "F_map": F_map.state_dict(),
            "F_style": F_style.state_dict(),
            "D": D.state_dict(),
            "opt_g": opt_g.state_dict(),
            "opt_d": opt_d.state_dict(),
            "iter": it,
            "CFG": CFG
        }, f"{CFG['out_dir']}/checkpoints/checkpoint_{it}.pth")

    # --- Affichage progression ---
    pbar.set_postfix({
        "D": f"{loss_d.item():.4f}", 
        "G": f"{loss_g.item():.4f}",
        "Cyc": f"{loss_cyc.item():.4f}",
        "Sty": f"{loss_sty.item():.4f}"
    })
    pbar.update(1)

# ====================== FIN TRAINING ======================
print("🎉 Training terminé !")

# --- Sauvegarde MODELE COMPLET pour AGE & YOUNG ---
torch.save({
    "G": G,
    "F_map": F_map, 
    "F_style": F_style,
    "D": D,
    "CFG": CFG,
    "target_domain": 2  # ÂGÉ
}, f"{CFG['out_dir']}/final_model_age.pth")

torch.save({
    "G": G,
    "F_map": F_map, 
    "F_style": F_style,
    "D": D,
    "CFG": CFG,
    "target_domain": 0  # JEUNE
}, f"{CFG['out_dir']}/final_model_young.pth")

print("✅ Modèles sauvegardés : final_model_age.pth & final_model_young.pth")

# ====================== Créer ZIP pour téléchargement facile (Kaggle) ======================
import shutil
shutil.make_archive(
    base_name=f"{CFG['out_dir']}/starganv2_models",
    format="zip",
    root_dir=CFG["out_dir"],
    base_dir="."
)
print("📦 Archive prête : starganv2_models.zip")

# ====================== TEST FINAL ======================
print("\n🧪 Test final des transformations...")
G.eval(); F_map.eval()
with torch.no_grad():
    test_img = real[0:1]
    source_domain = real_domain[0].item()
    print(f"Image source: {domain_names[source_domain]}")
    
    for target_domain in range(CFG["n_domains"]):
        if target_domain != source_domain:
            z = torch.randn(1, CFG["latent_dim"], device=device)
            style = F_map(z, torch.tensor([target_domain], device=device))
            result = G(test_img, style)
            result = add_gray_hair(result[0], torch.tensor(target_domain), CFG["gray_hair_intensity"]).unsqueeze(0)
            result = sharpen(result[0], CFG["sharpen_alpha"]).unsqueeze(0)
            save_image(result * 0.5 + 0.5, 
                      f"{CFG['out_dir']}/test_{domain_names[source_domain]}_to_{domain_names[target_domain]}.jpg")
            print(f"  → {domain_names[target_domain]} : sauvegardé")

print("✅ Tous les tests sont terminés !")
# ====================== FIN TRAINING ======================
print("🎉 Training terminé !")


# ====================== MODÈLE D'EXPORT POUR INFERENCE ======================
class StarGANv2AgeModel(nn.Module):
    def __init__(self, G, F_map, F_style, CFG):
        super().__init__()
        self.G = G
        self.F_map = F_map
        self.F_style = F_style
        self.CFG = CFG

    @torch.no_grad()
    def inference(self, img_tensor, target_domain, device):
        """
        img_tensor : (1, 3, 128, 128)
        target_domain : 0=jeune, 1=adulte, 2=âgé
        """
        z = torch.randn(1, self.CFG["latent_dim"], device=device)
        target_domain = torch.tensor([target_domain], device=device)
        style = self.F_map(z, target_domain)
        output = self.G(img_tensor.to(device), style)
        return output


# ====================== CRÉATION DES DEUX MODÈLES EXPORTABLES ======================
final_age_model = StarGANv2AgeModel(G, F_map, F_style, CFG)
final_young_model = StarGANv2AgeModel(G, F_map, F_style, CFG)


# ====================== SAUVEGARDE FORMAT FLASK ======================
torch.save(final_age_model, f"{CFG['out_dir']}/age_model.pth")
torch.save(final_young_model, f"{CFG['out_dir']}/young_model.pth")

print("🎉 Export propre terminé : age_model.pth et young_model.pth prêts à l’emploi !")


In [ ]:
# ====================== SAUVEGARDE FLASK COMPATIBLE ======================

class StarGANWrapper(nn.Module):
    def __init__(self, G, F_map, target_domain, cfg):
        super().__init__()
        self.G = G
        self.F_map = F_map
        self.target_domain = target_domain
        self.cfg = cfg

    def forward(self, x):
        b = x.size(0)
        z = torch.randn(b, self.cfg["latent_dim"]).to(x.device)
        domain = torch.full((b,), self.target_domain, device=x.device, dtype=torch.long)
        style = self.F_map(z, domain)
        return self.G(x, style)

# ----- MODELE AGE -----
age_model = StarGANWrapper(G, F_map, target_domain=2, cfg=CFG)
age_model.eval()

torch.save(age_model, f"{CFG['out_dir']}/final_model_age.pth")

# ----- MODELE JEUNE -----
young_model = StarGANWrapper(G, F_map, target_domain=0, cfg=CFG)
young_model.eval()

torch.save(young_model, f"{CFG['out_dir']}/final_model_young.pth")

print("✅ Modèles Flask prêts : final_model_age.pth et final_model_young.pth sauvegardés !")


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import io
import base64

# ====================== CLASSES ======================
class AdaIN(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.norm = nn.InstanceNorm2d(channels, affine=False)
        self.style = nn.Linear(style_dim, channels * 2)
    def forward(self, x, s):
        style = self.style(s)
        gamma, beta = style.chunk(2, 1)
        gamma = gamma.unsqueeze(-1).unsqueeze(-1)
        beta = beta.unsqueeze(-1).unsqueeze(-1)
        return gamma * self.norm(x) + beta

class ResBlock(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.conv2 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.adain1 = AdaIN(channels, style_dim)
        self.adain2 = AdaIN(channels, style_dim)
    def forward(self, x, s):
        h = self.conv1(x)
        h = self.adain1(h, s)
        h = F.relu(h, inplace=True)
        h = self.conv2(h)
        h = self.adain2(h, s)
        return x + h

class Generator(nn.Module):
    def __init__(self, style_dim=64, n_res=4):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv2d(3, 64, 7, 1, 3, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.ReLU(True),
        )
        self.res = nn.ModuleList([ResBlock(256, style_dim) for _ in range(n_res)])
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 3, 7, 1, 3), nn.Tanh()
        )
    def forward(self, x, s):
        h = self.enc(x)
        for block in self.res:
            h = block(h, s)
        return self.dec(h)

class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, n_domains=3):
        super().__init__()
        self.n_domains = n_domains
        self.shared = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(True),
            nn.Linear(256, 256), nn.ReLU(True),
        )
        self.branches = nn.ModuleList([
            nn.Sequential(
                nn.Linear(256, 256), nn.ReLU(True),
                nn.Linear(256, style_dim)
            ) for _ in range(n_domains)
        ])
    def forward(self, z, domain):
        h = self.shared(z)
        styles = [branch(h) for branch in self.branches]
        styles = torch.stack(styles, dim=1)
        batch_size = z.size(0)
        return styles[torch.arange(batch_size), domain]

# Wrapper pour utiliser directement
class StarGANWrapper(nn.Module):
    def __init__(self, G, F_map, target_domain, cfg):
        super().__init__()
        self.G = G
        self.F_map = F_map
        self.target_domain = target_domain
        self.cfg = cfg
    def forward(self, x):
        b = x.size(0)
        z = torch.randn(b, self.cfg["latent_dim"]).to(x.device)
        domain = torch.full((b,), self.target_domain, device=x.device, dtype=torch.long)
        style = self.F_map(z, domain)
        return self.G(x, style)

# ====================== DEVICE ======================
device = "cuda" if torch.cuda.is_available() else "cpu"

# ====================== CHEMINS ======================
young_model_path = "/kaggle/working/starganv2_optimized/final_model_young.pth"
age_model_path   = "/kaggle/working/starganv2_optimized/final_model_age.pth"

# ====================== CHARGEMENT ======================
young_model = torch.load(young_model_path, map_location=device, weights_only=False)
age_model   = torch.load(age_model_path, map_location=device, weights_only=False)

# ====================== FONCTIONS ======================
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

def decode_image(image_path):
    img = Image.open(image_path).convert("RGB")
    return transform(img).unsqueeze(0).to(device)

def encode_image(tensor):
    tensor = tensor.squeeze(0).cpu()
    tensor = (tensor * 0.5 + 0.5).clamp(0, 1)
    pil_img = transforms.ToPILImage()(tensor)
    buffer = io.BytesIO()
    pil_img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()

def transform_image(wrapper_model, image_path):
    wrapper_model.eval()
    img_tensor = decode_image(image_path)
    with torch.no_grad():
        out = wrapper_model(img_tensor)
    return encode_image(out)

# ====================== UTILISATION ======================
# Exemple :
# Résultat "jeune" à partir d'une image
img_base64 = transform_image(young_model, "/kaggle/working/cacd_cleaned_dataset_fast/trainC/AaronAshmore/55_AaronAshmore_0013.jpg")
# Résultat "âgé" à partir de la même image
img_base64_age = transform_image(age_model, "/kaggle/working/cacd_cleaned_dataset_fast/trainB/AaronPaul/28_AaronPaul_0001.jpg")

# img_base64 est en base64, tu peux l'afficher dans notebook par :
# from IPython.display import display, HTML
# display(HTML(f"<img src='data:image/png;base64,{img_base64}'/>"))


In [ ]:

# ====================== STAR GAN V2 - Version OPTIMISÉE (Netteté + Cheveux Gris + 40k iters) ======================
import os, random, glob
from collections import Counter
from tqdm import tqdm
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.utils import save_image
from torch.amp import autocast, GradScaler

# ====================== CONFIG OPTIMISÉE ======================
CFG = {
    "data_root": "/kaggle/working/cacd_complete_stars_dataset",
    "out_dir": "/kaggle/working/starganv2_optimized",
    "iters": 40000,  # +10k
    "img_size": 128,
    "batch_size": 28,
    "num_workers": 4,
    "lr": 1.5e-4,
    "latent_dim": 16,
    "style_dim": 64,
    "n_domains": 3,  # 0=jeune, 1=adulte, 2=âgé
    "n_res": 4,
    "lambda_cyc": 5.0,
    "lambda_cls": 1.0,
    "lambda_sty": 0.1,
    "lambda_reg": 0.0001,
    "sample_every": 500,
    "save_every": 5000,
    "seed": 42,
    "sharpen_alpha": 0.3,
    "gray_hair_intensity": 0.3
}

# ====================== SEED ======================
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(CFG["seed"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs(CFG["out_dir"] + "/samples", exist_ok=True)
os.makedirs(CFG["out_dir"] + "/checkpoints", exist_ok=True)

# ====================== UTILS ======================
def sharpen(img_tensor, alpha=0.3):
    kernel = torch.tensor([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=torch.float32, device=img_tensor.device)
    kernel = kernel.unsqueeze(0).unsqueeze(0)
    img_sharp = F.conv2d(img_tensor.unsqueeze(0), kernel.repeat(3,1,1,1), padding=1, groups=3)
    return torch.clamp((1-alpha)*img_tensor + alpha*img_sharp.squeeze(0), -1, 1)

def add_gray_hair(img_tensor, domain, intensity=0.3):
    if domain.item() == 2:  # domaine ÂGÉ
        gray = img_tensor.mean(dim=1, keepdim=True)
        img_tensor = img_tensor * (1-intensity) + gray * intensity
    return img_tensor

# ====================== DATASET ======================
class AgeDataset(Dataset):
    def __init__(self, root, img_size=128):
        self.samples = []
        self.domain_names = ["trainA", "trainB", "trainC"]
        for d_idx, folder in enumerate(self.domain_names):
            path = os.path.join(root, folder)
            if not os.path.isdir(path): continue
            for person in os.listdir(path):
                person_path = os.path.join(path, person)
                if not os.path.isdir(person_path): continue
                for file in glob.glob(os.path.join(person_path, "*.*")):
                    if file.lower().endswith((".jpg", ".jpeg", ".png", ".bmp")):
                        self.samples.append((file, d_idx))
        print(f"Dataset chargé : {len(self.samples)} images")
        print(f"Répartition des domaines: {Counter([d for _,d in self.samples])}")

        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, domain = self.samples[idx]
        try:
            img = Image.open(path).convert("RGB")
        except:
            img = Image.new("RGB", (CFG["img_size"], CFG["img_size"]))
        return self.transform(img), domain, path

# ====================== MODELS ======================
def weights_init(m):
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
        nn.init.normal_(m.weight, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            nn.init.constant_(m.bias, 0)

class AdaIN(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.norm = nn.InstanceNorm2d(channels, affine=False)
        self.style = nn.Linear(style_dim, channels * 2)
    def forward(self, x, s):
        style = self.style(s)
        gamma, beta = style.chunk(2, 1)
        gamma = gamma.unsqueeze(-1).unsqueeze(-1)
        beta = beta.unsqueeze(-1).unsqueeze(-1)
        return gamma * self.norm(x) + beta

class ResBlock(nn.Module):
    def __init__(self, channels, style_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.conv2 = nn.Conv2d(channels, channels, 3, 1, 1, bias=False)
        self.adain1 = AdaIN(channels, style_dim)
        self.adain2 = AdaIN(channels, style_dim)
    def forward(self, x, s):
        h = self.conv1(x)
        h = self.adain1(h, s)
        h = F.relu(h, inplace=True)
        h = self.conv2(h)
        h = self.adain2(h, s)
        return x + h

class Generator(nn.Module):
    def __init__(self, style_dim=64, n_res=4):
        super().__init__()
        self.enc = nn.Sequential(
            nn.Conv2d(3, 64, 7, 1, 3, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False), nn.ReLU(True),
        )
        self.res = nn.ModuleList([ResBlock(256, style_dim) for _ in range(n_res)])
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False), nn.ReLU(True),
            nn.Conv2d(64, 3, 7, 1, 3), nn.Tanh()
        )
    def forward(self, x, s):
        h = self.enc(x)
        for block in self.res:
            h = block(h, s)
        return self.dec(h)

class MappingNetwork(nn.Module):
    def __init__(self, latent_dim=16, style_dim=64, n_domains=3):
        super().__init__()
        self.n_domains = n_domains
        self.shared = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(True),
            nn.Linear(256, 256), nn.ReLU(True),
        )
        self.branches = nn.ModuleList([
            nn.Sequential(
                nn.Linear(256, 256), nn.ReLU(True),
                nn.Linear(256, style_dim)
            ) for _ in range(n_domains)
        ])
    def forward(self, z, domain):
        h = self.shared(z)
        styles = [branch(h) for branch in self.branches]
        styles = torch.stack(styles, dim=1)
        batch_size = z.size(0)
        return styles[torch.arange(batch_size), domain]

class StyleEncoder(nn.Module):
    def __init__(self, style_dim=64, n_domains=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 7, 1, 3), nn.ReLU(True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.ReLU(True),
            nn.Conv2d(128, 256, 4, 2, 1), nn.ReLU(True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.style = nn.Linear(256, style_dim)
        self.classifier = nn.Linear(256, n_domains)
    def forward(self, x):
        h = self.net(x).view(x.size(0), -1)
        style = self.style(h)
        cls_output = self.classifier(h)
        return style, cls_output

class Discriminator(nn.Module):
    def __init__(self, n_domains=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(128, 256, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(256, 512, 4, 2, 1), nn.LeakyReLU(0.2, True),
            nn.Conv2d(512, 1024, 4, 1, 0), nn.LeakyReLU(0.2, True),
        )
        self.src = nn.Conv2d(1024, 1, 3, 1, 1)
        self.cls = nn.Conv2d(1024, n_domains, 3, 1, 1)
    def forward(self, x):
        h = self.main(x)
        src = self.src(h).view(x.size(0), -1)
        cls = self.cls(h).mean([2,3])
        return src, cls

# ====================== DATA & MODELS ======================
dataset = AgeDataset(CFG["data_root"])
counts = Counter([d for _,d in dataset.samples])
weights = [1.0 / counts[d] for _,d in dataset.samples]
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
loader = DataLoader(dataset, batch_size=CFG["batch_size"], sampler=sampler,
                    num_workers=CFG["num_workers"], drop_last=True,
                    pin_memory=True, persistent_workers=True)

G = Generator(CFG["style_dim"], CFG["n_res"]).to(device)
F_map = MappingNetwork(CFG["latent_dim"], CFG["style_dim"], CFG["n_domains"]).to(device)
F_style = StyleEncoder(CFG["style_dim"], CFG["n_domains"]).to(device)
D = Discriminator(CFG["n_domains"]).to(device)

G.apply(weights_init); F_map.apply(weights_init); F_style.apply(weights_init); D.apply(weights_init)

opt_g = torch.optim.Adam(list(G.parameters()) + list(F_map.parameters()) + list(F_style.parameters()), 
                         lr=CFG["lr"], betas=(0.5, 0.999))
opt_d = torch.optim.Adam(D.parameters(), lr=CFG["lr"], betas=(0.5, 0.999))
scaler_g = GradScaler()
scaler_d = GradScaler()

# ====================== TRAINING LOOP ======================
pbar = tqdm(total=CFG["iters"])
domain_names = ["JEUNE", "ADULTE", "ÂGÉ"]

for it in range(1, CFG["iters"] + 1):
    real, real_domain, _ = next(iter(loader))
    real = real.to(device, non_blocking=True)
    real_domain = real_domain.to(device, non_blocking=True)
    b = real.size(0)

    # --- Discriminator ---
    opt_d.zero_grad()
    with autocast('cuda'):
        tgt_domain = torch.randint(0, CFG["n_domains"], (b,), device=device)
        z = torch.randn(b, CFG["latent_dim"], device=device)
        style = F_map(z, tgt_domain)
        fake = G(real, style).detach()

        real_src, real_cls = D(real)
        fake_src, _ = D(fake)

        loss_d_adv = F.softplus(-real_src).mean() + F.softplus(fake_src).mean()
        loss_d_cls = F.cross_entropy(real_cls, real_domain)
        loss_d = loss_d_adv + CFG["lambda_cls"] * loss_d_cls

    scaler_d.scale(loss_d).backward()
    scaler_d.step(opt_d)
    scaler_d.update()

    # --- Generator ---
    opt_g.zero_grad()
    with autocast('cuda'):
        tgt_domain = torch.randint(0, CFG["n_domains"], (b,), device=device)
        z = torch.randn(b, CFG["latent_dim"], device=device)
        style = F_map(z, tgt_domain)
        fake = G(real, style)
        fake_src, fake_cls = D(fake)

        loss_g_adv = -fake_src.mean()
        loss_g_cls = F.cross_entropy(fake_cls, tgt_domain)

        style_orig, _ = F_style(real)
        rec = G(fake, style_orig)
        loss_cyc = F.l1_loss(rec, real)

        z2 = torch.randn(b, CFG["latent_dim"], device=device)
        style2 = F_map(z2, tgt_domain)
        fake2 = G(real, style2)
        loss_sty = -F.l1_loss(fake, fake2) * CFG["lambda_sty"]

        style_reg = F.l1_loss(style, torch.zeros_like(style)) * CFG["lambda_reg"]

        loss_g = (loss_g_adv + CFG["lambda_cls"] * loss_g_cls + 
                 CFG["lambda_cyc"] * loss_cyc + loss_sty + style_reg)

    scaler_g.scale(loss_g).backward()
    scaler_g.step(opt_g)
    scaler_g.update()

    # --- SAMPLING DIAGNOSTIQUE ---
    if it % CFG["sample_every"] == 0:
        G.eval(); F_map.eval(); F_style.eval()
        with torch.no_grad():
            ref_images, ref_domains_list = [], []
            for domain in range(CFG["n_domains"]):
                domain_mask = (real_domain == domain)
                if domain_mask.sum() > 0:
                    idx = torch.where(domain_mask)[0][0]
                    ref_images.append(real[idx])
                    ref_domains_list.append(domain)
            
            if len(ref_images) >= 2:
                current_ref = torch.stack(ref_images[:min(4, len(ref_images))])
                ref_domains = torch.tensor(ref_domains_list[:current_ref.size(0)], device=device)
                
                outs = [current_ref.cpu()]
                
                for target_domain in range(CFG["n_domains"]):
                    tgt_domains = torch.full((current_ref.size(0),), target_domain, device=device)
                    z = torch.randn(current_ref.size(0), CFG["latent_dim"], device=device)
                    s = F_map(z, tgt_domains)
                    gen = G(current_ref, s)
                    # Appliquer cheveux gris et netteté
                    for i in range(gen.size(0)):
                        gen[i] = add_gray_hair(gen[i], tgt_domains[i], CFG["gray_hair_intensity"])
                        gen[i] = sharpen(gen[i], CFG["sharpen_alpha"])
                    outs.append(gen.cpu())

                grid = torch.cat(outs, dim=0)
                save_image(grid * 0.5 + 0.5, 
                          f"{CFG['out_dir']}/samples/{it:06d}.jpg", 
                          nrow=current_ref.size(0), padding=4, pad_value=1)
                
                ref_names = [domain_names[d] for d in ref_domains.cpu()]
                print(f"\n🎭 Sample {it}: {ref_names} → [JEUNE, ADULTE, ÂGÉ]")

        G.train(); F_map.train(); F_style.train()

       # --- Checkpoint (poids seulement pendant le training) ---
    if it % CFG["save_every"] == 0:
        torch.save({
            "G": G.state_dict(),
            "F_map": F_map.state_dict(),
            "F_style": F_style.state_dict(),
            "D": D.state_dict(),
            "opt_g": opt_g.state_dict(),
            "opt_d": opt_d.state_dict(),
            "iter": it,
            "CFG": CFG
        }, f"{CFG['out_dir']}/checkpoints/checkpoint_{it}.pth")

    # --- Affichage progression ---
    pbar.set_postfix({
        "D": f"{loss_d.item():.4f}", 
        "G": f"{loss_g.item():.4f}",
        "Cyc": f"{loss_cyc.item():.4f}",
        "Sty": f"{loss_sty.item():.4f}"
    })
    pbar.update(1)

# ====================== FIN TRAINING ======================
print("🎉 Training terminé !")

# --- Sauvegarde MODELE COMPLET (architecture + poids) ---
torch.save({
    "G": G,
    "F_map": F_map, 
    "F_style": F_style,
    "D": D,
    "CFG": CFG
}, f"{CFG['out_dir']}/final_model_complete_40k.pth")

print("✅ Modèle complet sauvegardé : final_model_complete_40k.pth")

# ====================== Créer ZIP pour téléchargement facile (Kaggle) ======================
import shutil

shutil.make_archive(
    base_name=f"{CFG['out_dir']}/starganv2_complete_model",
    format="zip",
    root_dir=CFG["out_dir"],
    base_dir="."
)

print("📦 Archive prête : starganv2_complete_model.zip")


# ====================== TEST FINAL ======================
print("\n🧪 Test final des transformations...")
G.eval(); F_map.eval()
with torch.no_grad():
    test_img = real[0:1]
    source_domain = real_domain[0].item()
    print(f"Image source: {domain_names[source_domain]}")
    
    for target_domain in range(CFG["n_domains"]):
        if target_domain != source_domain:
            z = torch.randn(1, CFG["latent_dim"], device=device)
            style = F_map(z, torch.tensor([target_domain], device=device))
            result = G(test_img, style)
            result = add_gray_hair(result[0], torch.tensor(target_domain), CFG["gray_hair_intensity"]).unsqueeze(0)
            result = sharpen(result[0], CFG["sharpen_alpha"]).unsqueeze(0)
            save_image(result * 0.5 + 0.5, 
                      f"{CFG['out_dir']}/test_{domain_names[source_domain]}_to_{domain_names[target_domain]}.jpg")
            print(f"  → {domain_names[target_domain]} : sauvegardé")

print("✅ Tous les tests sont terminés !")

Dataset chargé : 71191 images
Répartition des domaines: Counter({1: 51736, 2: 15862, 0: 3593})


  1%|▏         | 500/40000 [04:59<10:12:19,  1.08it/s, D=2.3837, G=1.4906, Cyc=0.1212, Sty=-0.0033]


🎭 Sample 500: ['JEUNE', 'ADULTE', 'ÂGÉ'] → [JEUNE, ADULTE, ÂGÉ]


  2%|▎         | 1000/40000 [09:53<6:35:54,  1.64it/s, D=2.3019, G=1.1347, Cyc=0.0908, Sty=-0.0016]


🎭 Sample 1000: ['JEUNE', 'ADULTE', 'ÂGÉ'] → [JEUNE, ADULTE, ÂGÉ]


...


🎭 Sample 39500: ['JEUNE', 'ADULTE', 'ÂGÉ'] → [JEUNE, ADULTE, ÂGÉ]


100%|█████████▉| 39999/40000 [6:28:18<00:00,  1.73it/s, D=1.3172, G=0.9196, Cyc=0.0565, Sty=-0.0002]


🎭 Sample 40000: ['JEUNE', 'ADULTE', 'ÂGÉ'] → [JEUNE, ADULTE, ÂGÉ]


100%|██████████| 40000/40000 [6:28:19<00:00,  1.45it/s, D=1.4058, G=1.2087, Cyc=0.0530, Sty=-0.0002]

🎉 Training terminé !
✅ Modèle complet sauvegardé : final_model_complete_40k.pth
